# Googleフォームで集めた情報を、自動スケジューリングソフトに流すための変換

このファイルのコピーを作ってそこで作業してください

In [1]:
import pandas as pd
import os
from src.solver import similar_name

In [2]:
# フォームで提出されたCSVファイルを読み込む
CSV_FILE_NAME = "sample.csv"
df = pd.read_csv(os.path.join("raw_data", CSV_FILE_NAME))
df.head()

,タイムスタンプ,メールアドレス,バンド名,メンバー1,メンバー2,メンバー3,メンバー4,メンバー5,メンバー6,出演可能時間,備考
0,2024/06/02 17:20:54,sample1@example.com,バンド1,アアアア,イイイイ,ウウウウ,エエエエ,オオオオ,カカカカ,"10:00-11:00,11:00-12:00,12:00-13:00,14:00-15:0...",いえーい
1,2024/06/02 17:47:16,sample2@example.com,バンド2,アアアア,イイイイ,キキキキ,クククク,ケケケケ,ココココ,"10:00-11:00, 11:00-12:00, 12:00-13:00",NaN
2,2024/06/02 17:56:35,sample3@example.com,バンド3,イイイイ,ササササ,シシシシ,セセセセ,ソソソソ,タタタタ,"10:00-11:00, 11:00-12:00, 12:00-13:00, 14:00-1...",NaN
3,2024/06/03 9:55:24,sample4@example.com,バンド4,チチチチ,ツツツツ,テテテテ,トトトト,ナナナナ,ニニニニ,"10:00-11:00, 11:00-12:00",NaN
4,2024/06/03 13:23:55,sample5@example.com,バンド5,ウウウウ,エエエエ,ヌヌヌヌ,ネネネネ,ノノノノ,ハハハハ,"10:00-11:00, 11:00-12:00, 12:00-13:00, 14:00-1...",NaN


## 出演可能時間なら1、不可能時間なら0となるように列を作る

In [3]:
# 出演可能時間でユニークな時間のリストを作成
unique_times = set()
for time in df["出演可能時間"]:
    time_list = time.split(",")
    for t in time_list:
        unique_times.add(t.strip())
unique_times = sorted(list(unique_times))
print(unique_times)

['10:00-11:00', '11:00-12:00', '12:00-13:00', '14:00-15:00', '15:00-16:00']


以下は実際のスケジュール (1時間あたり何バンドやるかなど)に合わせて書き換えて下さい

In [4]:
SCHEDULE_PER_TIME = 1
df_new = df.copy()
for time in unique_times:
    for i in range(SCHEDULE_PER_TIME):
        df_new[time + " " + str(i+1)] = [1 if time in [time_str.strip() for time_str in t.split(",")] else 0 for t in df["出演可能時間"]]
df_new = df_new.drop(columns=["出演可能時間"])
df_new.head()

,タイムスタンプ,メールアドレス,バンド名,メンバー1,メンバー2,メンバー3,メンバー4,メンバー5,メンバー6,備考,10:00-11:00 1,11:00-12:00 1,12:00-13:00 1,14:00-15:00 1,15:00-16:00 1
0,2024/06/02 17:20:54,sample1@example.com,バンド1,アアアア,イイイイ,ウウウウ,エエエエ,オオオオ,カカカカ,いえーい,1,1,1,1,1
1,2024/06/02 17:47:16,sample2@example.com,バンド2,アアアア,イイイイ,キキキキ,クククク,ケケケケ,ココココ,NaN,1,1,1,0,0
2,2024/06/02 17:56:35,sample3@example.com,バンド3,イイイイ,ササササ,シシシシ,セセセセ,ソソソソ,タタタタ,NaN,1,1,1,1,1
3,2024/06/03 9:55:24,sample4@example.com,バンド4,チチチチ,ツツツツ,テテテテ,トトトト,ナナナナ,ニニニニ,NaN,1,1,0,0,0
4,2024/06/03 13:23:55,sample5@example.com,バンド5,ウウウウ,エエエエ,ヌヌヌヌ,ネネネネ,ノノノノ,ハハハハ,NaN,1,1,1,1,1


備考に「最初の出番が良いです」と書いてある場合など、特別な配慮が必要な場合はここで編集すると良いでしょう。
Pandasで操作するのがしんどければ、一回CSVにしてダウンロードして、スプシで編集するのも手です。

## 名前が似ている人をあらかじめチェックしておく

バンドメンバーに関しては、文字列が完全一致した場合にのみ、同一人物とみなしています。誤植がある場合は別人とみなされてしまいます。似た名前の人を列挙し、単に似ているだけか、誤植か、見極めましょう。誤植の場合は直しましょう。

In [5]:
similar_name(os.path.join("raw_data", CSV_FILE_NAME), [3, 4, 5, 6, 7, 8])

[]

[]と表示されたら、似ている名前はありません。

## CSVとしてdataディレクトリに保存する

In [ ]:
if os.path.exists(os.path.join("data", CSV_FILE_NAME)):
    print("既にファイルが存在します")
else:
    df_new.to_csv(os.path.join("data", CSV_FILE_NAME), index=False)